## Introduction: Computationally Sampling Bradley-Terry

Given a preference interval $\mathbf u$ Bradley-Terry (BT) is a model for
drawing ballots when generating elections (TODO: insert link here).
In particular, `name_Bradley_Terry.generate_profile()` is a method
which, when given a preference interval, list of candidates and a number of ballots,
samples a profile of ballots according to the Bradley-Terry
distribution.  

In votekit today, sampling BT profiles is done by computing the probability mass function on
the set of ballots and then drawing from this pmf a single ballot at a
time. However, this involves computing $n!$ terms and
then storing all $n!$ probabilities. Using this method would mean that
Votekit is only able to support BT generative elections with up $11$
candidates. In this notebook we detail and investigate a Markov Chain
Monte Carlo approach which allows us to effectively simulate BT
elections with more than $30$ candidates and more than $1,000,000$
ballots.

### Imports and Helper functions

In [2]:
# adjust the directory so that we can import local source
import sys
import os

# Add the repo root to sys.path
ROOT_REL_PATH_FROM_NB = "../.."
repo_path = os.path.abspath(os.path.join(os.getcwd(), ROOT_REL_PATH_FROM_NB))  # adjust as needed
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

print(sys.path)

from src.votekit.ballot_generator import name_BradleyTerry 
from src.votekit.pref_interval import PreferenceInterval
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from math import factorial

['/home/willithg/MGGG/VoteKit', '/home/willithg/miniconda3/lib/python312.zip', '/home/willithg/miniconda3/lib/python3.12', '/home/willithg/miniconda3/lib/python3.12/lib-dynload', '', '/home/willithg/miniconda3/lib/python3.12/site-packages']


In [ ]:
def total_variation_distance(P, Q, labels):
    sum = 0 
    for x in labels:
        sum += abs(P[x] - Q[x])
    return (1/2)*sum

def generate_BT_args(n_cands, alpha=1):
    '''
        generates arguments for name_BradleyTerry class
        with n_cands, generates single slate and single bloc.
    '''
    if n_cands == 0:
        raise Exception("cannot generate BT args for 0 candidates")

    #cands = [chr(ord('a')+i) for i in range(n_cands)] # alphabetic candidates
    cands = [f"Cand{i}" for i in range(n_cands)]
    bloc_name = "H"
    pref_interval = PreferenceInterval.from_dirichlet(candidates=cands, alpha=alpha)
    pref_intervals_by_bloc = {
        bloc_name: {
            bloc_name : pref_interval
        }
    }
    bloc_voter_prop = {bloc_name: 1}
    cohesion_paramteters = {
        bloc_name: {
            bloc_name: 1
        }
    }
    
    return {
        "candidates": cands,
        "pref_intervals_by_bloc": pref_intervals_by_bloc,
        "bloc_voter_prop" : bloc_voter_prop,
        "cohesion_parameters": cohesion_paramteters
    }

def get_pdf_from_BT_instance(bt_inst):
    '''
        Given an instance of `name_BradleyTerry` this method returns
        the bt-pdf stored on the instance
    '''
    blocs = bt_inst.blocs
    if len(blocs) != 1:
        raise Exception("Non-one bloc on bt_inst")
    return bt_inst.pdfs_by_bloc[blocs[0]]

In [ ]:
bt = name_BradleyTerry(**generate_BT_args(n_cands=5, alpha=1))
pdf1 = get_pdf_from_BT_instance(bt)

prof = bt.generate_profile(number_of_ballots=10)
print(pdf1[tuple([list(i)[0] for i in prof.ballots[0].ranking])])
#prof.ballots[0].ranking


('Cand1', 'Cand3', 'Cand2', 'Cand0', 'Cand4')

## Comparing MCMC on Different Ballot Graphs

The method described in (TODO: link to general section) will also work
on any regular graph whose nodes are valid ballots. In particular
there are two commonly (citation?) used ballot graphs. Previously we
have defined two ballots to be adjacent if they differe by a single
neighbour swap. We can define the *shortcut ballot graph* by extending
this and defining two edges to be adjacent if they differ by a swap of
any two candidates. If our score function is the same as previously
defined then Metropolis-Hastings on the shortcut ballot graph still
has Bradley-Terry as a stationary distribution. 

In this section we will run some experiements comparing
Metropolois-Hastings on the two different kinds of Ballot graph.

### Comparing TV distance to BT Pdf

In the following cell we generate BT profiles by continuously sampled
MH on the shortcut and non-shortcut ballot graph. We do this on a
small enough set of candidates so that we can directly compute the
exact pdf. We then compare the TV distance between each sampled
profile to the known distribution.

In [ ]:
total_ballots = 10_000
num_ballots_array = np.linspace(100, total_ballots, 100)
num_ballots_array = num_ballots_array.astype(int)[1:]
num_trials_per_ballot_size = 10

N_CANDS_FOR_BALLOTS = 6
bt = name_BradleyTerry(**generate_BT_args(n_cands=N_CANDS_FOR_BALLOTS, alpha=1))
bt_pdfs = get_pdf_from_BT_instance(bt)

tv_distances_non_shortcut = []
tv_distances_shortcut = []
for num_ballots in tqdm(num_ballots_array):
    running_total_non_shortcut = 0
    running_total_shortcut = 0
    for _ in range(num_trials_per_ballot_size):
        ## -- Intialize the ballots ------------ 
        bals_BT_MCMC = bt.generate_profile_MCMC(num_ballots).group_ballots()
        bals_BT_MCMC_shortcut = bt.generate_profile_MCMC(num_ballots, on_shortcut_graph=True).group_ballots() 
        bals_BT_classic = bt.generate_profile(num_ballots).group_ballots()

        ## -- Calculate the frequencies ----------------
        ballot_rankings_MCMC = [bal.ranking for bal in bals_BT_MCMC.ballots]
        bal_labels_MCMC = ["".join([list(cand)[0] for cand in ranking]) for ranking in ballot_rankings_MCMC]
        bal_weights_MCMC = [int(bal.weight) for bal in bals_BT_MCMC.ballots]

        ballot_rankings_MCMC_shortcut = [bal.ranking for bal in bals_BT_MCMC_shortcut.ballots]
        bal_labels_MCMC_shortcut = ["".join([list(cand)[0] for cand in ranking]) for ranking in ballot_rankings_MCMC_shortcut]
        bal_weights_MCMC_shortcut = [int(bal.weight) for bal in bals_BT_MCMC_shortcut.ballots]
        
        ballot_rankings_classic = [bal.ranking for bal in bals_BT_classic.ballots]
        bal_labels_classic = ["".join([list(cand)[0] for cand in ranking]) for ranking in ballot_rankings_classic]
        bal_weights_classic = [int(bal.weight) for bal in bals_BT_classic.ballots]
        
        
        bal_rankings_map_classic_as_freq = {bal_labels_classic[i]: bal_weights_classic[i]/num_ballots for i in range(len(bal_labels_classic))}
        bal_rankings_map_MCMC = {bal_labels_MCMC[i]: bal_weights_MCMC[i]/num_ballots for i in range(len(bal_labels_MCMC))}
        bal_rankings_map_MCMC_shortcut_as_freq = {bal_labels_MCMC_shortcut[i]: bal_weights_MCMC_shortcut[i] / num_ballots for i in range(len(bal_labels_MCMC_shortcut))}

        # populate any missing keys
        # note: in general the classic map should have all the keys since
        # it computes the pdf directly. However, this procedure should
        # work in cases where we may not have the classic map
        all_keys = [list(m.keys()) for m in [
            bal_rankings_map_classic_as_freq, bal_rankings_map_MCMC, bal_rankings_map_MCMC_shortcut_as_freq
        ]]
        flattened_keys = [k for sublist in all_keys for k in sublist]

        for key in flattened_keys:
            bal_rankings_map_MCMC_shortcut_as_freq.setdefault(key, 0) 
            bal_rankings_map_MCMC.setdefault(key, 0) 
            bal_rankings_map_classic_as_freq.setdefault(key, 0) 

        # increment the recorded tv distances
        x_labels = list(bal_rankings_map_classic_as_freq.keys())
        running_total_non_shortcut += total_variation_distance(bal_rankings_map_MCMC, bal_rankings_map_classic_as_freq, x_labels)
        running_total_shortcut += total_variation_distance(bal_rankings_map_MCMC_shortcut_as_freq, bal_rankings_map_classic_as_freq, x_labels)

    tv_distances_non_shortcut.append(running_total_non_shortcut/num_trials_per_ballot_size)
    tv_distances_shortcut.append(running_total_shortcut/num_trials_per_ballot_size)